# LangSmith Tracing via the REST API

LangSmith tracing is built on "runs", which are analogous to traces and spans in OpenTelemetry.

If you aren't using a language supported by one of the LangSmith SDK's (python and JS/TS), then the only way to log chain and LLM runs to LangSmith is via this REST API.

### Prerequisites

A valid API key created in the LangSmith app

In [1]:
import os
from dotenv import load_dotenv


load_dotenv()

LANGSMITH_API_KEY = os.environ.get("LANGSMITH_API_KEY")
LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
project_name = "default"

### Understanding the Basics - Logging a Run
Each run represents the start and end of a function call, so we typically log the run in two calls, one to create the run and a second to end the run.

Events can be used to log additional minor information about what occurred during a run when that information doesn't merit an entire child run and is not the final output of the run.

**1. Create the Run**

Create the run by submitting a POST request at the beginning of the function call

In [2]:
import datetime
import uuid

import requests

run_id = str(uuid.uuid4())

res = requests.post(
    f"{LANGCHAIN_ENDPOINT}/runs",
    json={
        "id": run_id,
        "name": "MyFirstRun",
        "run_type": "chain",
        "start_time": datetime.datetime.utcnow().isoformat(),
        "inputs": {"text": "Foo"},
        "tags": ["langsmith", "rest", "my-example"],
    },
    headers={"x-api-key": LANGSMITH_API_KEY},
)

**Add events**

In [3]:
events = []
events.append({"event_name": "retry", "reason": "never gonna give you up"})
events.append({"event_name": "new_token", "value": "foo"})

**2. Update the Run**

Update the run via a PATCH request at the end.

In [4]:
requests.patch(
    f"{LANGCHAIN_ENDPOINT}/runs/{run_id}",
    json={
        "outputs": {"my_output": "Bar"},
        "end_time": datetime.datetime.utcnow().isoformat(),
        "events": events,
    },
    headers={"x-api-key": LANGSMITH_API_KEY},
)

<Response [200]>

### Logging an LLM Run

Correctly formatted runs with the run_type of "llm" let you:

- Track token usage
- Render "prettier" chat or completion message formats for better readability.

LangSmith supports OpenAI's llm message schema, so you can directly log the inputs and outputs of your call to any openai-compatible API without having to convert it to a new format.

**Logging LLM Chat Messages**

LangSmith expects the following format:

- Provide messages: [{"role": string, "content": string}] as a key-value pair in the inputs
- Provide choices: [{"message": {"role": string, "content": string}] as a key-value pair in the outputs.

In [5]:
run_id = str(uuid.uuid4())

res = requests.post(
    f"{LANGCHAIN_ENDPOINT}/runs",
    json={
        "id": run_id,
        "name": "MyChatModelRun",
        "run_type": "llm",
        "inputs": {
            "messages": [{"role": "user", "content": "What's the weather in SF like?"}],
            # Optional
            "model": "text-davinci-003",
            "functions": [
                {
                    "name": "get_current_weather",
                    "description": "Get the current weather in a given location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "unit": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                            },
                        },
                        "required": ["location"],
                    },
                }
            ],
            # You can add other invocation paramers as k-v pairs
            "temperature": 0.0,
        },
        "start_time": datetime.datetime.utcnow().isoformat(),
        "session_name": project_name,
    },
    headers={"x-api-key": LANGSMITH_API_KEY},
)

In [6]:
requests.patch(
    f"{LANGCHAIN_ENDPOINT}/runs/{run_id}",
    json={
        "end_time": datetime.datetime.utcnow().isoformat(),
        "outputs": {
            "choices": [
                {
                    "index": 0,
                    "message": {
                        "role": "assistant",
                        # Content is whatever string response the model generates
                        "content": "Mostly cloudy.",
                        # Function call is the function invocation and arguments as a string
                        "function_call": {
                            "name": "get_current_weather",
                            "arguments": '{\n"location": "San Francisco, CA"\n}',
                        },
                    },
                    "finish_reason": "function_call",
                }
            ],
        },
    },
    headers={"x-api-key": LANGSMITH_API_KEY},
)

<Response [200]>

### Nesting Runs

We can embed the nesting associations within the log, making it much easier to debug complex chains.

- Include a parent_run_id in your JSON body.
- Track the execution_order of the child run for it to be rendered correctly in the trace.

**RunLogger**

Create a new RunLogger class that manages the execution order state for us.

In [8]:
from typing import Optional

class RunLogger:
    def post_run(
        self, data: dict, name: str, run_id: str, parent_run_id: Optional[str] = None
    ) -> None:
        requests.post(
            f"{LANGCHAIN_ENDPOINT}/runs",
            json={
                "id": run_id,
                "name": name,
                "run_type": "chain",
                "parent_run_id": parent_run_id,
                "inputs": data,
                "start_time": datetime.datetime.utcnow().isoformat(),
                "session_name": project_name,
            },
            headers={"x-api-key": LANGSMITH_API_KEY},
        )

    def patch_run(
        self, run_id: str, output: Optional[dict] = None, error: Optional[str] = None
    ) -> None:
        requests.patch(
            f"{LANGCHAIN_ENDPOINT}/runs/{run_id}",
            json={
                "error": error,
                "outputs": output,
                "end_time": datetime.datetime.utcnow().isoformat(),
            },
            headers={"x-api-key": LANGSMITH_API_KEY},
        )

**Nested Run**

Create a simple fibonacci function and log each call as a "chain" run.

In [9]:
logger = RunLogger()

def fibonacci(n: int, depth: int = 0, parent_run_id: Optional[str] = None) -> int:
    run_id = str(uuid.uuid4())
    logger.post_run(
        {"n": n}, f"fibonacci_recursive", run_id, parent_run_id=parent_run_id
    )
    try:
        if n <= 1:
            result = n
        else:
            result = fibonacci(n - 1, depth + 1, parent_run_id=run_id) + fibonacci(
                n - 2, depth + 1, parent_run_id=run_id
            )
        logger.patch_run(run_id, output={"result": result})
        return result
    except Exception as e:
        logger.patch_run(run_id, error=str(e))
        raise

In [10]:
fibonacci(5)

5